In [ ]:
!pip install groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
from getpass import getpass

os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API KEY: ")

Enter your GROQ API KEY: ··········


In [ ]:
import os
from groq import Groq

# Initialize Groq client
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

BASE_MODEL = "llama-3.1-8b-instant"
# DEFINE EXPERT CONFIGURATIONS

MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a senior technical support engineer.
Be precise, rigorous, and code-focused.
Explain root causes clearly.
Provide code fixes when possible.
Avoid unnecessary fluff."""
    },
    "billing": {
        "system_prompt": """You are a billing support specialist.
Be empathetic and professional.
Focus on policies, refunds, subscriptions, and payment issues.
Explain next steps clearly and politely."""
    },
    "general": {
        "system_prompt": """You are a helpful customer support assistant.
Handle casual inquiries and general questions politely and clearly."""
    }
}


# ROUTER FUNCTION (temperature = 0)

def route_prompt(user_input: str) -> str:
    """
    Classifies the user input into:
    technical, billing, general, tool
    Returns ONLY the category name.
    """

    routing_prompt = """
Classify this customer query into ONE of these categories:

- technical (bugs, errors, programming issues)
- billing (refunds, charges, subscriptions, payments)
- general (casual chat or other inquiries)
- tool (real-time data like prices, current information)

Return ONLY the category name.
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,  # deterministic routing
        messages=[
            {"role": "system", "content": routing_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    category = response.choices[0].message.content.strip().lower()
    return category


# TOOL EXPERT

def get_bitcoin_price():
    """
    Mock tool function for Bitcoin price.
    In real life, you'd call a crypto API.
    """
    return "The current price of Bitcoin is approximately $62,500 (mock data)."


# ORCHESTRATOR

def process_request(user_input: str) -> str:
    """
    Main MoE orchestrator.
    """

    # Step 1: Route
    category = route_prompt(user_input)
    print(f"[Router Decision]: {category}")

    # Step 2: Tool handling (if needed)
    if category == "tool":
        if "bitcoin" in user_input.lower():
            return get_bitcoin_price()
        else:
            return "Tool request recognized, but no matching tool available."

    # Step 3: Load expert system prompt
    system_prompt = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])["system_prompt"]

    # Step 4: Call Expert LLM (temperature = 0.7)
    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content


# TEST INTERFACE

if __name__ == "__main__":
    while True:
        user_query = input("\nEnter your query (type 'exit' to quit): ")

        if user_query.lower() == "exit":
            break

        result = process_request(user_query)
        print("\n[Final Response]:")
        print(result)


Enter your query (type 'exit' to quit): What services do you offer?
[Router Decision]: general

[Final Response]:
Our company offers a wide range of services to cater to various needs. We have technical support for computer hardware and software, as well as assistance with digital products and subscriptions.

For those who need help with their devices, we offer repair services for smartphones, laptops, and other electronic equipment. Our team of experts can diagnose and fix issues related to software, hardware, and network connectivity.

We also provide support for internet and home network setup, including Wi-Fi configuration and troubleshooting. Additionally, we offer assistance with online security and virus removal.

If you have any questions or concerns about our services or need help with a specific issue, please feel free to ask and I'll be happy to assist you.

Enter your query (type 'exit' to quit): hi
[Router Decision]: general

[Final Response]:
Hello! How can I assist you 